In [ ]:
from zenml import step
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

@step
def local_data_loading() -> dict:
    """Load the iris dataset from scikit-learn."""
    iris = load_iris()
    X = iris.data
    y = iris.target
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    return {'X_train': X_train, 'X_test': X_test, 'y_train': y_train, 'y_test': y_test}


In [ ]:
from zenml import step
from sklearn.linear_model import LogisticRegression

@step(step_operator="k8s")
def remote_model_training(data: dict) -> LogisticRegression:
    """Train a logistic regression model on the iris dataset."""
    model = LogisticRegression()
    model.fit(data['X_train'], data['y_train'])
    
    train_accuracy = model.score(data['X_train'], data['y_train'])
    test_accuracy = model.score(data['X_test'], data['y_test'])
    
    print(f"Train accuracy: {train_accuracy:.2f}")
    print(f"Test accuracy: {test_accuracy:.2f}")
    
    return model

In [ ]:
from zenml import pipeline
from zenml.config import DockerSettings

@pipeline(settings={"docker": DockerSettings(parent_image="zenml:parent-image")})
def local_and_remote_pipeline():
    """Define a pipeline that connects the steps."""
    data = local_data_loading()
    model = remote_model_training(data)

In [ ]:
run = local_and_remote_pipeline()
print("success")

In [ ]:
print(run.steps["remote_model_training"].output.load())